In [1]:
using Oceananigans
using Oceananigans.Units

┌ Warning: You are using Julia v1.11 or later!"
│ Oceananigans is currently tested on Julia v1.10."
│ If you find issues with Julia v1.11 or later,"
│ please report at https://github.com/CliMA/Oceananigans.jl/issues/new
└ @ Oceananigans C:\Users\Tom Cummings\.julia\packages\Oceananigans\TXJRi\src\Oceananigans.jl:124


In [2]:
Lx = 1000kilometers # east-west extent [m]
Ly = 1000kilometers # north-south extent [m]
Lz = 1kilometers    # depth [m]

grid = RectilinearGrid(size = (48, 48, 8),
                       x = (0, Lx),
                       y = (-Ly/2, Ly/2),
                       z = (-Lz, 0),
                       topology = (Periodic, Bounded, Bounded))

48×48×8 RectilinearGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo
├── Periodic x ∈ [0.0, 1.0e6)          regularly spaced with Δx=20833.3
├── Bounded  y ∈ [-500000.0, 500000.0] regularly spaced with Δy=20833.3
└── Bounded  z ∈ [-1000.0, 0.0]        regularly spaced with Δz=125.0

In [ ]:


coriolis = BetaPlane(latitude = -45, f₀ = 1e-4)


α = 0

# Background deformation flow
Ū(x, y, z, t) = -α * x
V̄(x, y, z, t) = α*y  # No y-dependence in 2D

model = NonhydrostaticModel(
    grid = grid,
    coriolis = coriolis,
    buoyancy = BuoyancyTracer(),
    tracers = :b,
    background_fields = (u=Ū, v=V̄),  # Adds deformation flow
    advection = WENO(),              # High-order advection for fronts
    #closure = ScalarDiffusivity(ν=1e-5, κ=1e-5),
    )

LoadError: UndefVarError: `HydrostaticModel` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
"""
    ramp(y, Δy)

Linear ramp from 0 to 1 between -Δy/2 and +Δy/2.

For example:
```
            y < -Δy/2 => ramp = 0
    -Δy/2 < y < -Δy/2 => ramp = y / Δy
            y >  Δy/2 => ramp = 1
```
"""
ramp(y, Δy) = min(max(0, y/Δy + 1/2), 1)

N² = 1e-5 # [s⁻²] buoyancy frequency / stratification
M² = 1e-7 # [s⁻²] horizontal buoyancy gradient

Δy = 100kilometers # width of the region of the front
Δb = Δy * M²       # buoyancy jump associated with the front
ϵb = 1e-2 * Δb     # noise amplitude

bᵢ(x, y, z) = N² * z + Δb * ramp(y, Δy) + ϵb * randn()

set!(model, b=bᵢ)

In [ ]:
# Forcing terms (if not using background_fields)
#u_forcing(x, y, z, t, u, v, w, b) = α * u + coriolis.f₀ * v
#v_forcing(x, y, z, t, u, v, w, b) = -α * v - coriolis.f₀ * u

In [ ]:
simulation = Simulation(model, Δt=20minutes, stop_time=20days)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 20 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [ ]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [ ]:
using Printf

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(100))

Diagnostics/Output

In [ ]:
b = model.tracers.b
u, v, w = model.velocities
ζ = ∂x(v) - ∂y(u)
B = Average(b, dims=1)
U = Average(u, dims=1)
V = Average(v, dims=1)

filename = "frontogenesis"
save_fields_interval = 0.5day

slicers = (east = (grid.Nx, :, :),
           north = (:, grid.Ny, :),
           bottom = (:, :, 1),
           top = (:, :, grid.Nz))

for side in keys(slicers)
    indices = slicers[side]

    simulation.output_writers[side] = JLD2Writer(model, (; b, ζ);
                                                 filename = filename * "_$(side)_slice",
                                                 schedule = TimeInterval(save_fields_interval),
                                                 overwrite_existing = true,
                                                 indices)
end

simulation.output_writers[:zonal] = JLD2Writer(model, (; b=B, u=U, v=V);
                                               filename = filename * "_zonal_average",
                                               schedule = TimeInterval(save_fields_interval),
                                               overwrite_existing = true)

JLD2Writer scheduled on TimeInterval(12 hours):
├── filepath: frontogenesis_zonal_average.jld2
├── 3 outputs: (b, u, v)
├── array type: Array{Float32}
├── including: [:grid, :coriolis, :buoyancy, :closure]
├── file_splitting: NoFileSplitting
└── file size: 32.5 KiB

In [ ]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 1.574 minutes, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (36.488 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (9.763 seconds).


Visualization

In [ ]:
using CairoMakie

# Load data
b_xz = FieldTimeSeries(filename * "_xz_slice.jld2", "b")
ζ_xy = FieldTimeSeries(filename * "_xy_slice.jld2", "ζ")

# Plot final state
n = length(b_xz.times)

fig = Figure(size=(1200, 600))

ax1 = Axis(fig[1, 1], title="Buoyancy (xz slice)", xlabel="x (km)", ylabel="z (m)")
ax2 = Axis(fig[1, 2], title="Vorticity (surface)", xlabel="x (km)", ylabel="y (km)")

heatmap!(ax1, xnodes(b_xz)/1e3, znodes(b_xz), interior(b_xz[n], :, 1, :)')
heatmap!(ax2, xnodes(ζ_xy)/1e3, ynodes(ζ_xy), interior(ζ_xy[n], :, :, 1)')

Colorbar(fig[1, 3], heatmap!(ax1, ...), label="Buoyancy (m/s²)")
Colorbar(fig[1, 4], heatmap!(ax2, ...), label="Vorticity (s⁻¹)")

save("frontogenesis_final_state.png", fig)